In [ ]:
## without PCA

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, roc_auc_score, precision_recall_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory paths
train_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/train_data'
val_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/validation_data'
test_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/test_data'

# Image dimensions
img_height = 256
img_width = 256
batch_size = 16

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir,
                                                                       target_size=(img_height, img_width),
                                                                       batch_size=batch_size,
                                                                       class_mode='binary')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False)

# Linear Discriminant Analysis (LDA) Model
# Load the training data
train_images = []
train_labels = []
for images, labels in train_generator:
    train_images.append(images)
    train_labels.append(labels)
    if len(train_images) * batch_size >= len(train_generator.filenames):
        break
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

# Load the validation data
val_images = []
val_labels = []
for images, labels in val_generator:
    val_images.append(images)
    val_labels.append(labels)
    if len(val_images) * batch_size >= len(val_generator.filenames):
        break
val_images = np.concatenate(val_images)
val_labels = np.concatenate(val_labels)

# Load the test data
test_images = []
test_labels = []
for images, labels in test_generator:
    test_images.append(images)
    test_labels.append(labels)
    if len(test_images) * batch_size >= len(test_generator.filenames):
        break
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

# Flatten the image data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)
val_images_flattened = val_images.reshape(val_images.shape[0], -1)
test_images_flattened = test_images.reshape(test_images.shape[0], -1)

# Define and train the LDA model
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(train_images_flattened, train_labels)

# Predictions on validation set
val_pred_lda = lda_model.predict(val_images_flattened)

# Metrics on validation set
val_accuracy_lda = accuracy_score(val_labels, val_pred_lda)
val_precision_lda = precision_score(val_labels, val_pred_lda)
val_auc_lda = roc_auc_score(val_labels, lda_model.predict_proba(val_images_flattened)[:,1])

print("Linear Discriminant Analysis Metrics on Validation Set:")
print(f'Validation Accuracy: {val_accuracy_lda}')
print(f'Validation Precision: {val_precision_lda}')
print(f'Validation AUC: {val_auc_lda}')

# Predictions on test set
test_pred_lda = lda_model.predict(test_images_flattened)

# Metrics on test set
test_accuracy_lda = accuracy_score(test_labels, test_pred_lda)
test_precision_lda = precision_score(test_labels, test_pred_lda)
test_auc_lda = roc_auc_score(test_labels, lda_model.predict_proba(test_images_flattened)[:,1])

print("Linear Discriminant Analysis Metrics on Test Set:")
print(f'Test Accuracy: {test_accuracy_lda}')
print(f'Test Precision: {test_precision_lda}')
print(f'Test AUC: {test_auc_lda}')

# Compute probabilities for the test set
test_probabilities = lda_model.predict_proba(test_images_flattened)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(test_labels, test_probabilities)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Compute accuracy
accuracy = accuracy_score(test_labels, test_pred_lda)
print(f'Accuracy: {accuracy:.2f}')

# Compute precision
precision = precision_score(test_labels, test_pred_lda)
print(f'Precision: {precision:.2f}')

# Plot precision-recall curve
precision, recall, _ = precision_recall_curve(test_labels, test_probabilities)
plt.figure()
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:
##with PCA

import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# Directory paths
train_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/train_data'
val_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/validation_data'
test_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/test_data'

# Image dimensions
img_height = 256
img_width = 256
batch_size = 16

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir,
                                                                       target_size=(img_height, img_width),
                                                                       batch_size=1,  # Batch size set to 1
                                                                       class_mode='binary',
                                                                       shuffle=False)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=1,  # Batch size set to 1
                                                                        class_mode='binary',
                                                                        shuffle=False)

# Load the training data
train_images = []
train_labels = []
for images, labels in train_generator:
    train_images.append(images)
    train_labels.append(labels)
    if len(train_images) * batch_size >= len(train_generator.filenames):
        break
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

# Flatten the image data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)

# Apply PCA for feature reduction
pca = PCA(n_components=50)  # Adjust the number of components as needed
train_images_reduced = pca.fit_transform(train_images_flattened)

# Define and train the QDA model
qda_model = QuadraticDiscriminantAnalysis(reg_param=0.1)  # Adjust regularization parameter as needed
qda_model.fit(train_images_reduced, train_labels)

# Predictions on validation set
val_images, val_labels = [], []
for _ in range(len(val_generator.filenames)):
    images, labels = next(val_generator)  # Corrected line to fetch next batch
    val_images.append(images)
    val_labels.append(labels)
val_images = np.concatenate(val_images)
val_labels = np.concatenate(val_labels)
val_images_flattened = val_images.reshape(val_images.shape[0], -1)
val_images_reduced = pca.transform(val_images_flattened)
val_pred_qda = qda_model.predict(val_images_reduced)

# Metrics on validation set
val_accuracy_qda = accuracy_score(val_labels, val_pred_qda)
val_precision_qda = precision_score(val_labels, val_pred_qda)
val_auc_qda = roc_auc_score(val_labels, qda_model.predict_proba(val_images_reduced)[:,1])

print("Quadratic Discriminant Analysis Metrics on Validation Set:")
print(f'Validation Accuracy: {val_accuracy_qda}')
print(f'Validation Precision: {val_precision_qda}')
print(f'Validation AUC: {val_auc_qda}')

# Predictions on test set
test_images, test_labels = [], []
for _ in range(len(test_generator.filenames)):
    images, labels = next(test_generator)  # Corrected line to fetch next batch
    test_images.append(images)
    test_labels.append(labels)
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)
test_images_flattened = test_images.reshape(test_images.shape[0], -1)
test_images_reduced = pca.transform(test_images_flattened)
test_pred_qda = qda_model.predict(test_images_reduced)

# Metrics on test set
test_accuracy_qda = accuracy_score(test_labels, test_pred_qda)
test_precision_qda = precision_score(test_labels, test_pred_qda)
test_auc_qda = roc_auc_score(test_labels, qda_model.predict_proba(test_images_reduced)[:,1])

print("Quadratic Discriminant Analysis Metrics on Test Set:")
print(f'Test Accuracy: {test_accuracy_qda}')
print(f'Test Precision: {test_precision_qda}')
print(f'Test AUC: {test_auc_qda}')

# Compute probabilities for the test set
test_probabilities_qda = qda_model.predict_proba(test_images_reduced)[:, 1]

# Compute ROC curve and AUC
fpr_qda, tpr_qda, _ = roc_curve(test_labels, test_probabilities_qda)
roc_auc_qda = auc(fpr_qda, tpr_qda)

# Plot ROC curve
plt.figure()
plt.plot(fpr_qda, tpr_qda, color='darkorange', lw=2, label=f'AUC = {roc_auc_qda:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (QDA)')
plt.legend(loc='lower right')
plt.show()

# Compute precision-recall curve
precision_qda, recall_qda, _ = precision_recall_curve(test_labels, test_probabilities_qda)

# Plot precision-recall curve
plt.figure()
plt.plot(recall_qda, precision_qda, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (QDA)')
plt.show()
